Merge Datasates for Geolocation Analysis

liberaries

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipaddress




sys.path.append(os.path.abspath('../scripts'))
sys.path.append(os.path.abspath('..src'))

load data

In [2]:
fraud_data = pd.read_csv(r'C:\Users\befekadum\Documents\10x acadamy\week8and9\fraud_detection_system\data\Fraud_Data.csv')
credit_card_data = pd.read_csv(r'C:\Users\befekadum\Documents\10x acadamy\week8and9\fraud_detection_system\data\creditcard.csv')
ip_data = pd.read_csv(r'C:\Users\befekadum\Documents\10x acadamy\week8and9\fraud_detection_system\data\IpAddress_to_Country.csv')

In [14]:
ip_data

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216,16777471,Australia
1,16777472,16777727,China
2,16777728,16778239,China
3,16778240,16779263,Australia
4,16779264,16781311,China
...,...,...,...
138841,3758092288,3758093311,Hong Kong
138842,3758093312,3758094335,India
138843,3758095360,3758095871,China
138844,3758095872,3758096127,Singapore


Convert Ip addresses to integer format

In [15]:
# Convert IP address to integer format (removing the scientific notation)
fraud_data['ip_address'] = fraud_data['ip_address'].astype(int)


In [5]:
# Convert IP ranges in IP-to-Country dataset to integers
ip_data['lower_bound_ip_address'] = ip_data['lower_bound_ip_address'].astype(int)
ip_data['upper_bound_ip_address'] = ip_data['upper_bound_ip_address'].astype(int)


In [11]:
ip_data

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216,16777471,Australia
1,16777472,16777727,China
2,16777728,16778239,China
3,16778240,16779263,Australia
4,16779264,16781311,China
...,...,...,...
138841,3758092288,3758093311,Hong Kong
138842,3758093312,3758094335,India
138843,3758095360,3758095871,China
138844,3758095872,3758096127,Singapore


In [20]:

# Step 1: Create an interval index for the IP ranges in the IP-to-Country dataset
ip_ranges = pd.IntervalIndex.from_arrays(ip_data['lower_bound_ip_address'], 
                                         ip_data['upper_bound_ip_address'], 
                                         closed='both')  # Closed interval to include boundaries

# Step 2: Create a function to map the IP address to the corresponding country
def find_country(ip_address):
    # Find the index of the interval containing the IP address
    idx = ip_ranges.get_indexer([ip_address])
    
    # Return the country if a match is found, otherwise return NaN
    return ip_data['country'].iloc[idx[0]] if idx[0] != -1 else pd.NA

# Step 3: Apply this function to each IP address in the fraud dataset
fraud_data['country'] = fraud_data['ip_address'].apply(find_country)




   user_id          signup_time        purchase_time  purchase_value  \
0    22058  2015-02-24 22:55:49  2015-04-18 02:47:11              34   
1   333320  2015-06-07 20:39:50  2015-06-08 01:38:54              16   
2     1359  2015-01-01 18:52:44  2015-01-01 18:52:45              15   
3   150084  2015-04-28 21:13:25  2015-05-04 13:54:50              44   
4   221365  2015-07-21 07:09:52  2015-09-09 18:40:53              39   

       device_id source browser sex  age  ip_address  class        country  
0  QVPSPJUOCKZAR    SEO  Chrome   M   39   732758368      0          Japan  
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53   350311387      0  United States  
2  YSSKYOSJHPPLJ    SEO   Opera   M   53  2621473820      1  United States  
3  ATGTXKYKUDUQN    SEO  Safari   M   41  3840542443      0           <NA>  
4  NAUITBZFJKHWW    Ads  Safari   M   45   415583117      0  United States  


In [24]:
# Check the merged dataframe
fraud_data

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,732758368,0,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,350311387,0,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2621473820,1,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3840542443,0,<NA>
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,415583117,0,United States
...,...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,XPSKTWGPWINLR,SEO,Chrome,M,28,3451154526,1,United States
151108,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,LYSFABUCPCGBA,SEO,Safari,M,32,2439047221,0,Netherlands
151109,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,MEQHCSJUBRBFE,SEO,IE,F,26,2748470523,0,Japan
151110,207709,2015-07-09 20:06:07,2015-09-07 09:34:46,46,CMCXFGRHYSTVJ,SEO,Chrome,M,37,3601174708,0,United States
